# Astronomy 119 Final Project (#3): SEP Tutorial
## Astronomical Source Detection
### Fall 2018, 11:40am Section, Brant Robertson

### Authored by:
Laura Daniels (ladaniel@ucsc.edu)

Jennifer Bravo (jebravo@ucsc.edu)

Natalie Saechao (namsaech@ucsc.edu)

Simon Bukin (sbukin@ucsc.edu)


# Importing modules
In the following cells, we import the necessary modules for astronomical source detection.

In [ ]:
# Simon Bukin
import numpy as np
import sep
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

# Reading and displaying FITS Image
In this section, we read in the example FITS image and display it using matplotlib.

In [ ]:
# Simon Bukin
# open the data from the FITS image (this is a numpy array)
data = fits.open('image.fits')[0].data

# calculating mean and standard deviation
m, s = np.mean(data), np.std(data)
# displaying the image itself
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
# make sure to plot colorbar
plt.colorbar()
# Natalie Saechao
fig = plt.gcf()

In [ ]:
# Natalie Saechao
fig.savefig('image1_original.png')

# Background Subtraction
These cells deal with background subtraction of our initial image.

In [ ]:
# Simon Bukin
# measure the background (varying) of the image
bkg = sep.Background(data)
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# Simon Bukin
bkg_image = bkg.back()
# print the background subtracted image
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
# Natalie Saechao
fig = plt.gcf()

In [ ]:
# Natalie Saechao
fig.savefig('image2_background.png')

In [ ]:
# Simon Bukin
# calculate background noise as a 2d array
bkg_rms = bkg.rms()
# display the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
# don't forget the colorbar
plt.colorbar()
# Natalie Saechao
fig = plt.gcf()

In [ ]:
# Natalie Saechao
fig.savefig('image3_background_noise.png')

In [ ]:
# Simon Bukin
# data_sub is the initial data minus the background noise
data_sub = data - bkg

# Detecting Objects
Here, we detect the objects in our initial tests image using data_sub alongside SEP's extract function.

In [ ]:
# Simon Bukin
# use the SEP extract function to get all the detected opjects
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
# print out how many opjects were found
len(objects)

In [ ]:
# Simon Bukin
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
# caculate mean and standard deviation of subtracted data
m, s = np.mean(data_sub), np.std(data_sub)
# display the subtracted data
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# display each ellipse
for i in range(len(objects)):
    # new Ellipse crated for each object detected
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180 / np.pi)
    # no fill color
    e.set_facecolor('none')
    # red edge color
    e.set_edgecolor('red')
    ax.add_artist(e)

# Natalie Saechao
fig = plt.gcf()

In [ ]:
# Natalie Saechao
fig.savefig('image4_ellipse')

In [ ]:
# Simon Bukin
#all fields available for each object
objects.dtype.names

# Calculating Flux per Object
These cells deal with calculating the flux and fluxerr for every option.

In [ ]:
# Simon Bukin
# calculating the flux, fluxerr, and flag for each object
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# Simon Bukin
# first 10 results
for i in range(10):
    print('object: {:d}: flux= {:f} +/- {:f}'.format(i, flux[i], fluxerr[i]))